# Chi-Square Test of independence

In [1]:
import pandas as pd
import numpy as np
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve 
from copy import deepcopy
from scipy.stats import chi2_contingency

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure



In [2]:
df = pd.read_csv("Covid-19_dataset.csv",header=1)


df['is_Male'] = pd.get_dummies(df['Gender'])['MALE']#one hot encoding gender so male = #1 and female = 0

df = pd.concat([df, pd.get_dummies(df['Blood_Group'])], axis=1)
# df = pd.concat([df, pd.get_dummies(df['CONTROL_blood_group'])], axis=1) # needed to remove may not be needed
# df = pd.concat([df, pd.get_dummies(df['Risk_Factor'])], axis=1)
df = pd.concat([df, pd.get_dummies(df['Infection_type'])], axis=1)

blood_types = pd.get_dummies(df['Blood_Group']).columns


df = df.drop(['Age',' Incubation period (days)','Blood_Group','CONTROL_blood_group','Risk_Factor','Infection_type','Gender','Place_of_isolation','Patient_In','Patient_out'],axis=1)

columns = df.columns



# getting rid of blank values
print("before",len(df))
for name in columns:
    df = df[df[name] != " "]
    df = df.dropna()
    
label_Severeity = df['Severeity'] #Severeity vs Outcome
label_Outcome = df[["Outcome"]]
df = df.drop(['Severeity','Outcome'],axis=1)

df = df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))
columns = df.columns


print("after",len(df))

before 5668
after 5657


In [3]:
# map_mortality = {"Died":0,"Recovered":1}
# label_Outcome_mapped = deepcopy(label_Outcome)
# label_Outcome_mapped['Outcome']=label_Outcome_mapped['Outcome'].map(map_mortality)

In [4]:
# label_Outcome_mapped
map_key = {1:"Yes",0:"No"}

In [5]:
# label_Outcome_mapped
map_key = {1:"Yes",0:"No"}

for c in df.columns:
    df[c] = df[c].map(map_key)

In [6]:
df

,Fever,Chills,Cough,Dyspnea,Anosmia_Ageusia,Loss_of_appetite,Asthenia,Headache,Muscle_ache,Cyanosis,...,AB+,AB-,B+,B-,O+,O-,Asymptomatic,Asymtomatic,Symptomatic,Symtomatic
0,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,...,No,No,No,No,No,No,No,No,Yes,No
1,Yes,No,Yes,No,Yes,Yes,Yes,Yes,Yes,No,...,No,No,No,No,Yes,No,No,No,Yes,No
2,Yes,Yes,Yes,Yes,No,No,Yes,Yes,No,Yes,...,No,No,No,No,No,No,No,No,Yes,No
3,Yes,No,Yes,No,Yes,Yes,Yes,Yes,Yes,No,...,No,No,No,No,No,No,No,No,Yes,No
4,No,No,Yes,No,Yes,Yes,Yes,Yes,Yes,No,...,No,No,Yes,No,No,No,No,No,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5663,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,...,No,No,Yes,No,No,No,No,No,Yes,No
5664,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,No,No,Yes,No,No,No,No,No,Yes,No
5665,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,...,No,No,Yes,No,No,No,No,No,Yes,No
5666,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,No,No,No,No,No,No,No,No,Yes,No


In [7]:
def rank(predictor):
    X = pd.DataFrame(df[predictor])
    X['Outcome'] = label_Outcome['Outcome']

    contingency_table = pd.crosstab(X['Outcome'], X[predictor])

    # Apply chi-square test
    chi2, p_value, dof, expected_freq = chi2_contingency(contingency_table)
    
    
    return chi2, p_value, dof, expected_freq



list_of_corr = []

for feature in df.columns:
    chi2, p_value, dof, expected_freq = rank(feature)
    list_of_corr.append({"Feature":feature, "Chi" : chi2,'p_value': p_value})

    

df_corr = pd.DataFrame(list_of_corr)
# df_abs = pd.DataFrame(abs_corr)
#https://towardsdatascience.com/feature-selection-with-boruta-in-python-676e3877e596

In [8]:
df_corr = df_corr.sort_values(by=['Chi','Feature'], ascending=False)
df_corr

,Feature,Chi,p_value
9,Cyanosis,278.732065,1.418742e-62
17,AB+,4.124114,4.227616e-02
16,A-,3.941346,4.711305e-02
12,Diarrhea,3.737087,5.321716e-02
2,Cough,2.690450,1.009513e-01
11,Sore_throat,2.686511,1.012012e-01
1,Chills,1.906072,1.674003e-01
18,AB-,1.806348,1.789469e-01
14,is_Male,1.727771,1.886958e-01
3,Dyspnea,1.513177,2.186550e-01
